In [1]:
!pip install pandas

In [ ]:
import pandas as pd

# อ่านข้อมูลจากไฟล์ CSV
df = pd.read_csv(r"dataforecast\raw_data\POWER_Point_Hourly_20210316_20250316_009d14N_099d31E_LST.csv") # นำเข้าfilepathเองของ powerpointhourly
df.drop(columns=['PS'], inplace=True)
# ลบค่าซ้ำ (ถ้ามี)
df_cleaned = df.drop_duplicates()

# ตรวจหาค่าผิดปกติ (outliers) โดยใช้ IQR
Q1 = df_cleaned.quantile(0.25)
Q3 = df_cleaned.quantile(0.75)
IQR = Q3 - Q1

# กำหนดขอบเขตค่าปกติ
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# กรองเฉพาะค่าที่อยู่ในช่วงปกติ
df_cleaned = df_cleaned[~((df_cleaned < lower_bound) | (df_cleaned > upper_bound)).any(axis=1)]

# แปลงคอลัมน์ YEAR, MO, DY, HR เป็นคอลัมน์ DATE
df_cleaned["DATE"] = pd.to_datetime(
    df_cleaned['YEAR'].astype(str) + '-' +
    df_cleaned['MO'].astype(str).str.zfill(2) + '-' +
    df_cleaned['DY'].astype(str).str.zfill(2) + ' ' +
    df_cleaned['HR'].astype(str).str.zfill(2) + ':00:00',
    format='%Y-%m-%d %H:%M:%S'
)

# Drop the old columns
df_cleaned = df_cleaned.drop(columns=["YEAR", "MO", "DY", "HR"])

# Save the updated file
df_cleaned.to_csv("cleaned_data_with_datetime.csv", index=False)

# Display the first few rows
cleaned_file_path = r"dataforecast\raindata.csv" # ส่งออกไฟล์ path เอง
df_cleaned.to_csv(cleaned_file_path, index=False)
df_cleaned

,T2M,RH2M,WS10M,PRECTOTCORR,WSC,DATE
0,23.51,92.91,1.94,0.00,1.28,2021-03-16 00:00:00
1,23.31,93.30,1.69,0.00,1.14,2021-03-16 01:00:00
2,23.08,93.99,1.57,0.00,1.07,2021-03-16 02:00:00
3,22.82,94.89,1.62,0.00,1.10,2021-03-16 03:00:00
4,22.54,96.13,1.76,0.00,1.22,2021-03-16 04:00:00
...,...,...,...,...,...,...
35011,25.87,90.32,1.96,0.42,1.67,2025-03-13 19:00:00
35012,25.53,90.78,1.90,1.12,1.48,2025-03-13 20:00:00
35013,25.30,91.08,1.83,0.06,1.38,2025-03-13 21:00:00
35014,25.19,90.62,1.68,0.06,1.22,2025-03-13 22:00:00
